# Data Scientist

<div class="alert alert-info">
Before running this notebook, start a PyGrid Domain. For a local instance using Docker you can use <code>scripts/start_grid_domain.sh</code>.
</div>

In [1]:
from typing import Any

import torch as T

import syft as sy
from syft import make_plan
from syft.federated.model_centric_fl_client import ModelCentricFLClient
from syft.lib.python.int import Int
from syft.lib.python.list import List

## Step 1: Specify configuration

In [2]:
NAME = "dqn"
VERSION = "0.0.0"
GRID_ADDRESS = "localhost:5000"

CLIENT_CONFIG = {
    "name": NAME,
    "version": VERSION,
    "alpha": 0.001,
    "gamma": 0.99,
    "min_epsilon": 0.01,
    "epsilon_reduction": 0.0001,
    "n_train_iterations": 1000,
    "n_test_iterations": 100,
    # "max_updates": 1,  # custom syft.js option that limits number of training loops per worker
}

SERVER_CONFIG = {
    "min_workers": 2,
    "max_workers": 2,
    "pool_selection": "random",
    "do_not_reuse_workers_until_cycle": 6,
    "cycle_length": 28800,  # max cycle length in seconds
    "num_cycles": 30,  # max number of cycles
    "max_diffs": 1,  # number of diffs to collect before avg
    "minimum_upload_speed": 0,
    "minimum_download_speed": 0,
    "iterative_plan": True,  # tells PyGrid that avg plan is executed per diff
}

INPUT_WIDTH = 4
HIDDEN_WIDTH = 4
OUTPUT_WIDTH = 2

## Step 2: Define the model to be hosted

In [3]:
class DQNAgent(sy.Module):
    def __init__(self, torch_ref: Any, input_width, output_width, hidden_width, initial_epsilon=1.0) -> None:
        super().__init__(torch_ref=torch_ref)
        nn = torch_ref.nn
        self.network = nn.Sequential(
            nn.Linear(input_width, hidden_width),
            nn.ReLU(),
            nn.Linear(hidden_width, output_width),
        )
        # HACK: We can’t transfer parameters except via a PyTorch Module
        self.epsilon = nn.Linear(1, 1, bias=False)
        for p in self.epsilon.parameters():
            p.requires_grad = False
        self.epsilon.weight.data = torch_ref.tensor([initial_epsilon], requires_grad=False).data

In [4]:
local_agent = DQNAgent(T, INPUT_WIDTH, OUTPUT_WIDTH, HIDDEN_WIDTH)

## Step 3: Define the training and averaging plans

In [5]:
# Hack: we need a client plan so PyGrid will allow our client to download the model
# We can't use a training plan right now because the Blackjack env isn't supported yet
@make_plan
def nothing_plan():
    pass

@make_plan
def averaging_plan(
    # Average of diffs, not parameters
    current_average=List(local_agent.parameters()),
    next_diff=List(local_agent.parameters()),
    num=Int(0),
):
    return [
        (current_param * num + diff_param) / (num + 1)
        for current_param, diff_param in zip(current_average, next_diff)
    ]

## Step 4: Host in PyGrid

In [6]:
grid = ModelCentricFLClient(address=GRID_ADDRESS, secure=False)
grid.connect()

In [7]:
response = grid.host_federated_training(
    model=local_agent,
    client_plans={"nothing_plan": nothing_plan},
    client_protocols={},
    server_averaging_plan=averaging_plan,
    client_config=CLIENT_CONFIG,
    server_config=SERVER_CONFIG,
)
print(f"Host response: {response}")

Host response: {'type': 'model-centric/host-training', 'data': {'status': 'success'}}


<div class="alert alert-info">
Now run the Data Owner notebook to train the model.
</div>